In [1]:
import pandas as pd
import numpy as np
import re
import os

In [2]:
# Create a path to output obtained data to the weather folder in the raw_data folder
raw_data_path = '../raw_data/weather'

# Check files in directory
weather_file_lst = os.listdir(raw_data_path)
weather_file_lst

['aberporth.txt',
 'armagh.txt',
 'ballypatrick.txt',
 'bradford.txt',
 'braemar.txt',
 'camborne.txt',
 'cambridge.txt',
 'cardiff.txt',
 'chivenor.txt',
 'cwmystwyth.txt',
 'dunstaffnage.txt',
 'durham.txt',
 'eastbourne.txt',
 'eskdalemuir.txt',
 'heathrow.txt',
 'hurn.txt',
 'lerwick.txt',
 'leuchars.txt',
 'lowestoft.txt',
 'manston.txt',
 'nairn.txt',
 'newtonrigg.txt',
 'oxford.txt',
 'paisley.txt',
 'ringway.txt',
 'rossonwye.txt',
 'shawbury.txt',
 'sheffield.txt',
 'southampton.txt',
 'stornoway.txt',
 'suttonbonington.txt',
 'tiree.txt',
 'valley.txt',
 'waddington.txt',
 'whitby.txt',
 'wickairport.txt',
 'yeovilton.txt']

## ETL and Data Consolidation

In [3]:
def get_lastpos_header(headers_lst, line):
    '''
    Get the last characters' position in headers_lst in the line
    headers_lst (list): list of headers
    line (string): the line which contain headers
    '''
    line_copy = line
    
    last_pos_lst = []
    
    for header in headers_lst:
        if re.search(header, line_copy) is not None:
            pos = re.search(header, line_copy).span()
            last_pos = pos[1]
            
            # Update line by replacing those found with ' '
            line_list = list(line_copy)
            line_list[pos[0]: pos[1]] = [' ']*(pos[1]-pos[0])
            line_copy = ''.join(line_list)
            
            # Update last_pos_lst
            last_pos_lst.append(last_pos)
            
        else:
            print('Unknown Error while processing {}'.format(header))
            break
            
    return last_pos_lst

In [4]:
def match_headerlines(header1_lst, header1_lastpos_lst, header2_lst, header2_lastpos_lst):
    '''
    Function that takes in 4 arguments: header1_lst, header1_lastpos_lst, header2_lst, header2_lastpos_lst
    and matches the last positions of the headers in the first header line with the last positions of the headers in the
    2nd header line; for those that could be matched, it implies that the header in the 2nd header line actually a wrapped
    text from the corresponding header in the first header line. Thus, I concatenate these headers and return them
    header1_lst (list): list containing the headers from the first header line
    header1_lastpos_lst(list): list containing headers' last positions from the first header line
    header2_lst (list): list containing the headers from the 2nd header line
    header2_lastpos_lst(list): list containing headers' last positions from the 2nd header line
    '''
    
    for header1_index, header1_lastpos in enumerate(header1_lastpos_lst):
        
        # Header 2 index match
        if header1_lastpos in header2_lastpos_lst:
            header2_index_match = header2_lastpos_lst.index(header1_lastpos)

            # Clean up the header1
            header1_lst[header1_index] = header1_lst[header1_index] + ' ' + header2_lst[header2_index_match]

        
    return header1_lst
    

In [5]:
def load_and_proc_weather_file(weather_file):
    
    '''
    Loads and processes the weather file after taking the name of the weather file as input
    weather_file (string): name of the station's weather data file.
    '''
    
    stn_dir = os.path.join(raw_data_path, weather_file)

    with open(stn_dir, 'r') as text_file:
        text = text_file.read()
        
#     print('Loading {}'.format(stn_dir))

    # Using the assumption that the first word must be the station name,
    # the row corresponding to matches to strings: 'yyyy', 'mm', 'tmax', 'tmin', 'af', 'rain', 'sun' must be the header row
    # and that the header row will wrap to the row below.
    # Assume that spacing is consistent between one field and next, so the last character's position of each header will match
    # the last character's position of the header that has been wrapped to the next line

    # Check if first header has been found
    header_flag = False

    # list to store actual data
    actual_data_lst = []
    
    # Initialise lat, lon
    lat, lon = (None, None)

    for line_num, line in enumerate(text.splitlines()):

            
        # Get the longitude and latitude
        result = re.findall('Lat (-?\d+\.*\d+) Lon (-?\d+\.*\d*),?\w?',line)

        if len(result) > 0:
            lat, lon = result[0]
            #print(line)

        if header_flag == False:

            # Check for header row
            header_lst = ['yyyy', 'mm', 'tmax', 'tmin', 'af', 'rain', 'sun']
            check = all([header in line for header in header_lst])

            if check == True:

                # Get header line index
                header_line_num = line_num

                # Split this by space for this current line
                header1 = line.split()

                # Get the next line and do this as well
                line2 = text.splitlines()[line_num+1]
                header2 = line2.split()

                # Get the position of last character for each header
                header1_lastpos_lst = get_lastpos_header(header1, line)
                header2_lastpos_lst = get_lastpos_header(header2, line2)

                # Match and combine the headers
                combined_header = match_headerlines(header1, header1_lastpos_lst, header2, header2_lastpos_lst)

                header_flag = True

        else:
            # Adjust for 2nd row of headerline
            if line_num == header_line_num + 1:
                continue
            else:
                # Update data, keeping only the first few list entries corresponding to the the headers
                data = line.split()
                data = data[:len(combined_header)]
                actual_data_lst.append(data)

    # Create a pandas dataframe to store this data
    df = pd.DataFrame(actual_data_lst)

    # Assign column headers
    df.columns = combined_header

    # Add in the station name
    stn_name = weather_file.split('.')[0]
    df['station'] = stn_name
    
    # Add latitude and longitude
    df['lat'] = lat
    df['lon'] = lon
    
    print('{} processed successfully'.format(stn_name))

    return df 

In [6]:
# Process the weather data files and store these dataframes as a list
weather_df_lst = [load_and_proc_weather_file(weatherfile) for weatherfile in weather_file_lst]

aberporth processed successfully
armagh processed successfully
ballypatrick processed successfully
bradford processed successfully
braemar processed successfully
camborne processed successfully
cambridge processed successfully
cardiff processed successfully
chivenor processed successfully
cwmystwyth processed successfully
dunstaffnage processed successfully
durham processed successfully
eastbourne processed successfully
eskdalemuir processed successfully
heathrow processed successfully
hurn processed successfully
lerwick processed successfully
leuchars processed successfully
lowestoft processed successfully
manston processed successfully
nairn processed successfully
newtonrigg processed successfully
oxford processed successfully
paisley processed successfully
ringway processed successfully
rossonwye processed successfully
shawbury processed successfully
sheffield processed successfully
southampton processed successfully
stornoway processed successfully
suttonbonington processed success

In [7]:
# Concat them together for further pre-procesing and cleaning
weather_df = pd.concat(weather_df_lst)
weather_df = weather_df.reset_index(drop = True)
weather_df

,yyyy,mm,tmax degC,tmin degC,af days,rain mm,sun hours,station,lat,lon
0,1941,1,---,---,---,74.7,---,aberporth,52.139,-4.570
1,1941,2,---,---,---,69.1,---,aberporth,52.139,-4.570
2,1941,3,---,---,---,76.2,---,aberporth,52.139,-4.570
3,1941,4,---,---,---,33.7,---,aberporth,52.139,-4.570
4,1941,5,---,---,---,51.3,---,aberporth,52.139,-4.570
...,...,...,...,...,...,...,...,...,...,...
38071,2020,11,12.8,5.7,6,59.8,62.5#,yeovilton,51.006,-2.641
38072,2020,12,9.3*,2.6*,9*,95.7*,59.6*,yeovilton,51.006,-2.641
38073,2021,1,7.6*,0.8*,16*,76.1*,43.2*,yeovilton,51.006,-2.641
38074,2021,2,9.5,2.4,11,61.8,65.6#,yeovilton,51.006,-2.641


## Clean each column

In [8]:
# The year field should only be 4 digits and be integer

# Upon investigation, there are cases where Site closed takes up a row of data, we will need to clean this first
weather_df['yyyy'][weather_df['yyyy'].apply(lambda x: len(re.findall('\d{4}',x)) ==0)]

8987     Site
24390    Site
29780    Site
Name: yyyy, dtype: object

In [9]:
# Drop the rows where year is equal to site
weather_df = weather_df[weather_df['yyyy'] != 'Site']

In [10]:
# Cleaning the year column
weather_df['yyyy'] = weather_df['yyyy'].apply(lambda x: int(re.findall('\d{4}',x)[0]))

C:\Users\angsi\anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
# Clean the mm column
weather_df['mm'] = weather_df['mm'].apply(lambda x: int(re.findall('\d{1,2}',x)[0]) if len(re.findall('\d{1,2}',x)) > 0 else np.nan)

C:\Users\angsi\anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
# Use a regex to file the digits with decimal places
regex = re.compile('\-?\d+\.*\d*')

In [13]:
# Clean each of the following columns accordingly
num_cols = ['tmax degC', 'tmin degC', 'af days', 'rain mm', 'sun hours']
for num_col in num_cols:
    weather_df[num_col] = weather_df[num_col].apply(lambda x: float(re.findall(regex, str(x))[0]) if len(re.findall(regex, str(x))) > 0 else np.nan)
    print('{} processed successfully'.format(num_col))

C:\Users\angsi\anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


tmax degC processed successfully
tmin degC processed successfully
af days processed successfully
rain mm processed successfully
sun hours processed successfully


In [14]:
# Do some random checks
weather_df[(weather_df['yyyy'] == 1942) & (weather_df['station'] == 'aberporth')]

,yyyy,mm,tmax degC,tmin degC,af days,rain mm,sun hours,station,lat,lon
12,1942,1,5.8,2.1,NaN,114.0,58.0,aberporth,52.139,-4.570
13,1942,2,4.2,-0.6,NaN,13.8,80.3,aberporth,52.139,-4.570
14,1942,3,9.7,3.7,NaN,58.0,117.9,aberporth,52.139,-4.570
15,1942,4,13.1,5.3,NaN,42.5,200.1,aberporth,52.139,-4.570
16,1942,5,14.0,6.9,NaN,101.1,215.1,aberporth,52.139,-4.570
17,1942,6,16.2,9.9,NaN,2.3,269.3,aberporth,52.139,-4.570
18,1942,7,17.4,11.3,NaN,70.2,185.0,aberporth,52.139,-4.570
19,1942,8,18.7,12.3,NaN,78.5,141.9,aberporth,52.139,-4.570
20,1942,9,16.4,10.7,NaN,146.8,129.1,aberporth,52.139,-4.570
21,1942,10,13.1,8.2,NaN,131.1,82.1,aberporth,52.139,-4.570


In [15]:
# Clean lat and lon
weather_df['lat'] = weather_df['lat'].astype(float)
weather_df['lon'] = weather_df['lon'].astype(float)

C:\Users\angsi\anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\angsi\anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## Export for Further Analysis

In [16]:
# Export to proc_data for further analysis
proc_data_path = '..\proc_data\weather'

weather_df.to_csv(os.path.join(proc_data_path,'weather.csv'), index = False)